<a href="https://colab.research.google.com/github/vaniabordin/desafio/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Célula 1 — Instalação de dependências

In [ ]:
# Executar (célula com "Code")
!pip install -q --upgrade pip
!pip install -q git+https://github.com/openai/whisper.git   # whisper
!pip install -q gradio==3.39.0                              # interface web
!pip install -q transformers                               # Hugging Face pipelines
!pip install -q yake                                       # keyphrase extraction
!pip install -q gTTS                                       # text-to-speech (Google)
!pip install -q soundfile                                  # leitura/saída de áudio


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Célula 2 — Imports e carregamento rápido de modelos

In [ ]:
import os
import whisper
import gradio as gr
from transformers import pipeline
import yake
from gtts import gTTS
import uuid
import soundfile as sf
from IPython.display import Audio, display

# Modelo whisper:small para arquivos maiores e mais precisos
WHISPER_MODEL = "small"

print("Carregando Whisper modelo:", WHISPER_MODEL)
model = whisper.load_model(WHISPER_MODEL)

# Pipeline de sentimento (multilíngue)
sentiment_model = pipeline("sentiment-analysis",
                           model="nlptown/bert-base-multilingual-uncased-sentiment")

# Configuração YAKE para extração de keywords
yake_kw_extractor = yake.KeywordExtractor(lan="pt", n=3, top=8)  # linguagem pt, ajustar se precisar


Carregando Whisper modelo: small


Device set to use cpu


Célula 3 — Funções utilitárias: transcrição, sentimento, keywords, TTS

In [ ]:
def transcribe_audio(audio_filepath, lang=None):
    # whisper espera arquivo de áudio (wav/mp3) - Colab/Gradio salva em filepath
    # lang opcional: 'pt'
    options = {}
    if lang:
        options['language'] = lang
    # Transcribe retorna dict com 'text'
    result = model.transcribe(audio_filepath, **options)
    text = result.get("text", "").strip()
    return text

def analyze_sentiment(text):
    # Usa o model nlptown — retorna estrelas (1-5)
    try:
        out = sentiment_model(text[:512])  # limita para evitar excesso
        return out  # lista com dicionários contendo label e score
    except Exception as e:
        return [{"label":"ERROR","score":0.0, "error": str(e)}]

def extract_keywords(text, lang="pt"):
    if not text or len(text.strip())==0:
        return []
    # YAKE requer ajustar linguagem;
    keywords = yake_kw_extractor.extract_keywords(text)
    # keywords é lista de tuples (keyword, score); retorna só keywords ordenadas
    return [kw for kw,score in keywords]

def text_to_speech(text, lang="pt"):
    if not text or len(text.strip()) == 0:
        return None
    filename = f"tts_{uuid.uuid4().hex}.mp3"  # salva na pasta atual
    tts = gTTS(text=text, lang=lang)
    tts.save(filename)
    return filename

#Texto para audio
texto ="A análise de dados é um processo sistêmico para inspecionar, limpar, transformar e modelar dados brutos."
arquivo = text_to_speech(texto)
print("Arquivo salvo em:", arquivo)

# Reproduz o áudio
display(Audio(arquivo, autoplay=True))


Arquivo salvo em: tts_e918f61275374242ab93782149ebb7b9.mp3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Célula 4 — Função principal que integra tudo (para Gradio)

In [ ]:
def process_audio(audio_file):
    # audio_file: caminho para arquivo salvo pelo Gradio (type="filepath")
    if audio_file is None:
        return "Nenhum áudio recebido.", None, [], None

    # Transcrição (detecção automática de idioma)
    transcript = transcribe_audio(audio_file, lang=None)

    # Análise de sentimento
    sentiment = analyze_sentiment(transcript)

    # Extração de keywords
    keywords = extract_keywords(transcript)

    # Gerar TTS (retorna um arquivo para o usuário ouvir a sintetização da transcrição)
    # Define lang='pt' por padrão.
    tts_file = text_to_speech(transcript, lang="pt")

    return transcript, sentiment, keywords, tts_file


Célula 5 — Interface Gradio (UI)

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Laboratório de Fala — Colab Sandbox\nGrave com o microfone abaixo e veja transcrição, sentimento e keywords.")

    with gr.Row():
        audio_input = gr.Audio(label="Grave sua fala", type="filepath")  # removido source
        btn = gr.Button("Enviar")

    with gr.Row():
        out_transcript = gr.Textbox(label="Transcrição")

    with gr.Row():
        out_sentiment = gr.JSON(label="Sentimento (pipeline)")
        out_keywords = gr.Textbox(label="Keywords (YAKE)")

    with gr.Row():
        out_tts = gr.Audio(label="Text-to-Speech (síntese da transcrição)")

    def wrapper(audio):
        transcript, sentiment, keywords, tts_file = process_audio(audio)
        kw_str = ", ".join(keywords) if keywords else ""
        return transcript, sentiment, kw_str, tts_file

    btn.click(fn=wrapper, inputs=[audio_input], outputs=[out_transcript, out_sentiment, out_keywords, out_tts])

demo.launch(share=True)



/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:93: UserWarning: IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade. 
--------
  ).start()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57105009b2100f1b3e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
